# rubin_nights demo

The rubin_nights package provides simple utilties to accessing the EFD, the ConsDB and various logging reports. 


In [1]:
import os
from astropy.time import Time, TimeDelta
from IPython.display import Markdown, display, HTML

from rubin_nights import connections

## Gather connection information and authorization

In the USDF (usdf-rsp or usdf-rsp-dev) or summit RSP, the default is to connect to the services appropriate to the same RSP.

Outside of these RSPs, the default is to connect to the production USDF services at usdf-rsp.slac.stanford.edu, although the usdf-rsp-dev is available as well. Note that outside of the RSP, an RSP token will be required. See https://rsp.lsst.io/v/usdfprod/guides/auth/creating-user-tokens.html for more information about creating tokens (and note that tokens created on usdf-rsp are different than those on usdf-rsp-dev and must match the services being queried). 

In [2]:
on_rsp = False
# Are you on an RSP?
if on_rsp:
    tokenfile = None
    site = None
# Or are you outside of an RSP? - just use USDF and your own USDF-RSP token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
else:
    # Substitute the location of your own tokenfile
    # If you prefer, this will also get token info from an "ACCESS_TOKEN" environment variable
    tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
    site = 'usdf'

endpoints = connections.get_clients(tokenfile = tokenfile, site=site)

The services available and their endpoints:

In [3]:
endpoints

{'api_base': 'https://usdf-rsp.slac.stanford.edu',
 'efd': efd at https://usdf-rsp.slac.stanford.edu/influxdb-enterprise-data,
 'obsenv': lsst.obsenv at https://usdf-rsp.slac.stanford.edu/influxdb-enterprise-data,
 'too': lsst.scimma at https://summit-lsp.lsst.codes/influxdb,
 'dm': lsst.dm at https://usdf-rsp-dev.slac.stanford.edu/influxdb,
 'consdb': https://usdf-rsp.slac.stanford.edu/consdb,
 'consdb_tap': https://usdf-rsp.slac.stanford.edu/api/consdbtap,
 'narrative_log': https://usdf-rsp.slac.stanford.edu/narrativelog/messages,
 'exposure_log': https://usdf-rsp.slac.stanford.edu/exposurelog/messages,
 'night_report': https://usdf-rsp.slac.stanford.edu/nightreport/reports}

In [4]:
# Example of choosing to connect to the usdf-rsp-dev services instead
endpoints_dev = connections.get_clients(tokenfile=os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp"), site='usdf-dev')
endpoints_dev

{'api_base': 'https://usdf-rsp-dev.slac.stanford.edu',
 'efd': efd at https://usdf-rsp.slac.stanford.edu/influxdb-enterprise-data,
 'obsenv': lsst.obsenv at https://usdf-rsp.slac.stanford.edu/influxdb-enterprise-data,
 'too': lsst.scimma at https://summit-lsp.lsst.codes/influxdb,
 'dm': lsst.dm at https://usdf-rsp-dev.slac.stanford.edu/influxdb,
 'consdb': https://usdf-rsp-dev.slac.stanford.edu/consdb,
 'consdb_tap': https://usdf-rsp-dev.slac.stanford.edu/api/consdbtap,
 'narrative_log': https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages,
 'exposure_log': https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages,
 'night_report': https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports}

## Querying for data

Each of these services has different data. A basic query setup along the lines of "what happened on a given night" is included with most of the connection classes, and demonstrated below. 
Each connection also supports more general queries, `endpoints["xxx"].query` but the details of what to include to specify such a query depends on the service. 

More information: 
* [night report](https://usdf-rsp.slac.stanford.edu/nightreport/docs#/default/find_nightreports_reports_get)
* [narrative log](https://usdf-rsp.slac.stanford.edu/narrativelog/docs#/default/find_messages_messages_get)
* [exposure log](https://usdf-rsp.slac.stanford.edu/exposurelog/docs#/default/find_messages_messages_get)
* [Consdb Schema](https://sdm-schemas.lsst.io/) - see also [consdb technote](https://dmtn-227.lsst.io/)
    * [consdb FastAPI](https://usdf-rsp.slac.stanford.edu/consdb/docs#/default/query_consdb_query_post)
    * [consdb_TAP](https://pyvo.readthedocs.io/en/latest/dal/index.html#pyvo-tap)
* [EFD Schema](https://ts-xml.lsst.io/) - see also [lsst-efd-client documentation](https://efd-client.lsst.io/index.html)


In [5]:
# Let's pick a day_obs to query 
# day_obs = the local calendar date of the start (sunset) of a night
day_obs = "2025-04-15"
# the day_obs is applied to the entire time from noon UTC to next-noon
t_start = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
t_end = t_start + TimeDelta(1, format='jd')
t_start.iso, t_end.iso

('2025-04-15 12:00:00.000', '2025-04-16 12:00:00.000')

---
    
### Night report

The night report is a summary of the observing state. There is one for each telescope. 

In [6]:
nightreport, html = endpoints['night_report'].query_night_report(day_obs=day_obs, telescope="Simonyi", return_html=True)
display(HTML(html))

---

### Narrative log

The narrative log is an ongoing record of comments regarding observing and the observatory state.
When paired with further information about the scriptqueue, it provides valuable insights into observatory operations.

In [7]:
log = endpoints['narrative_log'].query_log(t_start, t_end)
# The log is indexed by time such that it can be joined with EFD data.
display(HTML(log[['component', 'message_text', 'user_id']].iloc[21:35].to_html()))

---

### Exposure log

The exposure log provides a record for annotations on particular images. It tends to be very sparsely populated.

In [8]:
exposurelog = endpoints['exposure_log'].query_log(t_start, t_end)
# Exposure log needs to be joined with other data sources (exposure id <-> exposure time) in order to have a useful "time" value. 
exposurelog

Empty DataFrame
Columns: []
Index: []

---

### ConsDB

The consolidated database is still in development. For more information on contents, see [sdm-schemas](https://sdm-schemas.lsst.io/). 
For more information on development, see also [consdb technote](https://dmtn-227.lsst.io/). 

The TAP service is intended to be the future API for users, and is currently available on all USDF platforms (and summit?). 
The FastAPI service is available on at the USDF as well as the summit. This interface is intended to be deprecated in favor of TAP, but does perform better presently.

See also the ConsDB demo notebook in this repo. 

In [9]:
# Let's pick a different day_obs to query - 2025-04-15 is nice but many of the visits were missing metadata
# day_obs = the local calendar date of the start (sunset) of a night
day_obs = "2025-06-20"
# the day_obs is applied to the entire time from noon UTC to next-noon
t_start = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
t_end = t_start + TimeDelta(1, format='jd')
t_start.iso, t_end.iso

('2025-06-20 12:00:00.000', '2025-06-21 12:00:00.000')

In [10]:
visits_api = endpoints['consdb'].get_visits('lsstcam', t_start, t_end, augment=False)
visits_tap = endpoints['consdb_tap'].get_visits('lsstcam', t_start, t_end, augment=False)
print(len(visits_api), len(visits_tap))

681 681


In [11]:
short_cols = ['obs_start', 'obs_start_mjd', 's_ra', 's_dec', 'sky_rotation', 'band', 
         'airmass', 'psf_sigma_median', 'zero_point_median', 'img_type', 'target_name', 'observation_reason', 'science_program']
if len(visits_api) == 0:
    print("Did not find any visits")
else:
    display(visits_api[short_cols].iloc[200:210])

obs_start  obs_start_mjd        s_ra      s_dec  \
200  2025-06-21T02:00:57.212000   60847.083996  255.473934 -40.759790   
201  2025-06-21T02:01:36.146000   60847.084446  253.818986 -40.992684   
202  2025-06-21T02:02:13.759000   60847.084881  252.153094 -41.201750   
203  2025-06-21T02:02:52.532000   60847.085330  255.657066 -40.732457   
204  2025-06-21T02:03:32.228000   60847.085790  251.859087 -41.479587   
205  2025-06-21T02:04:11.080000   60847.086239  255.379445 -41.015921   
206  2025-06-21T02:04:50.024000   60847.086690  253.718456 -41.248968   
207  2025-06-21T02:05:29.262000   60847.087144  252.045707 -41.457923   
208  2025-06-21T02:06:08.304000   60847.087596  255.563388 -40.988558   
209  2025-06-21T02:06:47.833000   60847.088054  251.281166 -41.335570   

     sky_rotation band   airmass  psf_sigma_median  zero_point_median  \
200    199.572470    r  1.096767          2.138978          32.139519   
201    200.925032    r  1.086554          2.247843          32.149834   
202    202.417681    r  1.077204          2.360815          32.178200   
203    196.319362    r  1.094851          2.332669          32.131207   
204    201.410813    r  1.074847          2.282093          32.172848   
205    195.198142    r  1.091935          2.264779          32.131302   
206    196.587486    r  1.082199          2.131186          32.145775   
207    198.119773    r  1.073272          2.038430          32.169750   
208    191.940821    r  1.090050          2.032248          32.137920   
209    196.855129    r  1.067627          2.132667          32.179882   

    img_type target_name    observation_reason science_program  
200  science       Prawn  field_survey_science       BLOCK-365  
201  science       Prawn  field_survey_science       BLOCK-365  
202  science       Prawn  field_survey_science       BLOCK-365  
203  science       Prawn  field_survey_science       BLOCK-365  
204  science       Prawn  field_survey_science       BLOCK-365  
205  science       Prawn  field_survey_science       BLOCK-365  
206  science       Prawn  field_survey_science       BLOCK-365  
207  science       Prawn  field_survey_science       BLOCK-365  
208  science       Prawn  field_survey_science       BLOCK-365  
209  science       Prawn  field_survey_science       BLOCK-365

In [12]:
from rubin_nights.augment_visits import augment_visits

In [13]:
query = "select v.*, q.zero_point_median, q.sky_bg_median from cdb_lsstcomcam.visit1 as v, cdb_lsstcomcam.visit1_quicklook as q"
query += " where q.visit_id = v.visit_id and v.science_program = 'BLOCK-320'"
print(query)
# Either of these methods will work - query issues are more clear through TAP
#visits_lsstcomcam = endpoints['consdb_tap'].query(query)
visits_lsstcomcam = endpoints['consdb'].query(query)
visits_lsstcomcam = augment_visits(visits_lsstcomcam, "lsstcomcam")
print(f"Retrieved {len(visits_lsstcomcam)} visits") # should retrieve 1694 visits
cols = ['obs_start', 'obs_start_mjd', 's_ra', 's_dec', 'sky_rotation', 'band', 'exp_time', 'zero_point_median', 'sky_bg_median',
         'airmass', 'img_type', 'target_name', 'observation_reason', 'science_program', 'visit_gap', 'clouds']
visits_lsstcomcam[cols].tail()

select v.*, q.zero_point_median, q.sky_bg_median from cdb_lsstcomcam.visit1 as v, cdb_lsstcomcam.visit1_quicklook as q where q.visit_id = v.visit_id and v.science_program = 'BLOCK-320'
Retrieved 1694 visits


obs_start  obs_start_mjd        s_ra      s_dec  \
1690  2024-12-12T08:27:55.306000   60656.352723  106.262882 -10.700520   
1691  2024-12-12T08:28:29.033000   60656.353114  106.272920 -10.496384   
1680  2024-12-12T08:29:11.158000   60656.353601  106.360054 -10.420939   
1692  2024-12-12T08:29:44.543000   60656.353988  106.394204 -10.512472   
1693  2024-12-12T08:30:18.670000   60656.354383  106.398960 -10.505149   

      sky_rotation band  exp_time  zero_point_median  sky_bg_median   airmass  \
1690     90.220293    r      30.0          31.965206    2187.837891  1.217461   
1691     90.458045    r      30.0          31.975727    2263.498779  1.220842   
1680    114.567933    r      30.0          31.976316    2409.519287  1.222558   
1692    114.490657    r      30.0          31.957592    2554.209229  1.222920   
1693    114.502141    r      30.0          31.967522    2693.477783  1.224567   

     img_type target_name observation_reason science_program  visit_gap  \
1690  science     Seagull            science       BLOCK-320   3.226597   
1691  science     Seagull            science       BLOCK-320   3.293132   
1680  science     Seagull            science       BLOCK-320  11.685077   
1692  science     Seagull            science       BLOCK-320   2.948549   
1693  science     Seagull            science       BLOCK-320   3.701174   

        clouds  
1690 -0.028641  
1691 -0.039575  
1680 -0.040374  
1692 -0.021694  
1693 -0.031825

---

### EFD

There is a full client for the EFD (see [efd-lsst-client](https://efd-client.lsst.io/)) but a simple, synchronous query client is provided here that replicates the basics of the API.
Finding information in the EFD can be challenging; see [the XML specifications](https://ts-xml.lsst.io/) - after identifying the CSC that is the source of the information, the commands and logentry information can be linked to the specifications for each topic.

In [14]:
targets = endpoints['efd'].select_time_series('lsst.sal.Scheduler.logevent_target', '*', t_start, t_end, index=1)
targets.tail()

airmass  alt  az  blockId  cloud      decl  \
time                                                                            
2025-06-21 08:55:17.521931+00:00  1.111979    0   0   106123      0 -22.94919   
2025-06-21 08:55:30.610939+00:00  1.184870    0   0   106124      0 -22.46952   
2025-06-21 10:07:57.472851+00:00  0.000000    0   0   106145      0   0.00000   
2025-06-21 10:28:07.793130+00:00  0.000000    0   0   106153      0   0.00000   
2025-06-21 10:35:40.713032+00:00  0.000000    0   0   106161      0   0.00000   

                                  exposureTimes0  exposureTimes1  \
time                                                               
2025-06-21 08:55:17.521931+00:00              30               0   
2025-06-21 08:55:30.610939+00:00              30               0   
2025-06-21 10:07:57.472851+00:00               0               0   
2025-06-21 10:28:07.793130+00:00               0               0   
2025-06-21 10:35:40.713032+00:00               0               0   

                                  exposureTimes2  exposureTimes3  ...  \
time                                                              ...   
2025-06-21 08:55:17.521931+00:00               0               0  ...   
2025-06-21 08:55:30.610939+00:00               0               0  ...   
2025-06-21 10:07:57.472851+00:00               0               0  ...   
2025-06-21 10:28:07.793130+00:00               0               0  ...   
2025-06-21 10:35:40.713032+00:00               0               0  ...   

                                  skyBrightness  slewTime  \
time                                                        
2025-06-21 08:55:17.521931+00:00      20.230215     120.0   
2025-06-21 08:55:30.610939+00:00      20.170797     120.0   
2025-06-21 10:07:57.472851+00:00       0.000000       0.0   
2025-06-21 10:28:07.793130+00:00       0.000000       0.0   
2025-06-21 10:35:40.713032+00:00       0.000000       0.0   

                                                                        snapshotUri  \
time                                                                                  
2025-06-21 08:55:17.521931+00:00  https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...   
2025-06-21 08:55:30.610939+00:00  https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...   
2025-06-21 10:07:57.472851+00:00                                                      
2025-06-21 10:28:07.793130+00:00                                                      
2025-06-21 10:35:40.713032+00:00                                                      

                                  solarElong  sunAlt  sunAz sunDec  sunRa  \
time                                                                        
2025-06-21 08:55:17.521931+00:00           0       0      0      0      0   
2025-06-21 08:55:30.610939+00:00           0       0      0      0      0   
2025-06-21 10:07:57.472851+00:00           0       0      0      0      0   
2025-06-21 10:28:07.793130+00:00           0       0      0      0      0   
2025-06-21 10:35:40.713032+00:00           0       0      0      0      0   

                                  targetId  targetName  
time                                                    
2025-06-21 08:55:17.521931+00:00       409     lowdust  
2025-06-21 08:55:30.610939+00:00       411       bulgy  
2025-06-21 10:07:57.472851+00:00         0      Target  
2025-06-21 10:28:07.793130+00:00         0      Target  
2025-06-21 10:35:40.713032+00:00         0      Target  

[5 rows x 63 columns]

In [15]:
prev_targets = endpoints['efd'].select_top_n('lsst.sal.Scheduler.logevent_target', '*', num=3, time_cut=t_start)
prev_targets

SchedulerID   airmass  alt  az  blockId  \
time                                                                       
2025-06-20 11:52:15.348317+00:00        None  1.407627    0   0   301926   
2025-06-20 11:42:15.427953+00:00        None  1.407627    0   0   301924   
2025-06-20 11:32:15.343744+00:00        None  1.407627    0   0   301922   

                                  cloud     decl  exposureTimes0  \
time                                                               
2025-06-20 11:52:15.348317+00:00      0  14.7556              30   
2025-06-20 11:42:15.427953+00:00      0  14.7556              30   
2025-06-20 11:32:15.343744+00:00      0  14.7556              30   

                                  exposureTimes1  exposureTimes2  ...  \
time                                                              ...   
2025-06-20 11:52:15.348317+00:00               0               0  ...   
2025-06-20 11:42:15.427953+00:00               0               0  ...   
2025-06-20 11:32:15.343744+00:00               0               0  ...   

                                  skyBrightness  slewTime  \
time                                                        
2025-06-20 11:52:15.348317+00:00              2       120   
2025-06-20 11:42:15.427953+00:00              2       120   
2025-06-20 11:32:15.343744+00:00              2       120   

                                                                        snapshotUri  \
time                                                                                  
2025-06-20 11:52:15.348317+00:00  https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...   
2025-06-20 11:42:15.427953+00:00  https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...   
2025-06-20 11:32:15.343744+00:00  https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...   

                                  solarElong  sunAlt  sunAz  sunDec sunRa  \
time                                                                        
2025-06-20 11:52:15.348317+00:00           0       0      0       0     0   
2025-06-20 11:42:15.427953+00:00           0       0      0       0     0   
2025-06-20 11:32:15.343744+00:00           0       0      0       0     0   

                                  targetId  targetName  
time                                                    
2025-06-20 11:52:15.348317+00:00        12   SimTarget  
2025-06-20 11:42:15.427953+00:00        11   SimTarget  
2025-06-20 11:32:15.343744+00:00        10   SimTarget  

[3 rows x 66 columns]